# Suffix Arrays and BWT/FM-Index for Exact Pattern Matching

This notebook demonstrates the implementation and performance of suffix arrays and BWT/FM-index for exact pattern matching in strings.

## 1. Introduction

### Suffix Arrays

A **suffix array** is a sorted array of all suffixes of a string. It's a space-efficient alternative to suffix trees that enables fast pattern matching.

For a text `T` of length `n`, the suffix array `SA[0..n-1]` contains the starting positions of all suffixes sorted in lexicographic order.

### Burrows-Wheeler Transform (BWT)

The **BWT** is a reversible transformation that permutes the characters of a text to make it more compressible. It's also the foundation of the FM-index.

### FM-Index

The **FM-index** (Full-text index in Minute space) is a compressed data structure that enables fast pattern matching using the BWT, C-table, and Occ table.

**Time Complexities:**
- Suffix Array Construction (Prefix Doubling): **O(n log² n)**
- SA Pattern Search: **O(m log n + k)** where m is pattern length, k is occurrences
- BWT Construction: **O(n)**
- C-table Construction: **O(n + σ)** where σ is alphabet size
- Occ table Construction: **O(σn)**
- FM Backward Search: **O(m + k)**

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from src.sa import build_suffix_array
from src.sa_search import find_all as sa_find_all
from src.bwt import (
    build_bwt,
    build_c_table,
    build_occ_table,
    fm_find_all
)
import time
import random
import matplotlib.pyplot as plt
import numpy as np

## 2. Basic Examples

Let's start with a simple example to understand how suffix arrays and BWT work.

In [ ]:
# Example text
text = "banana$"

# Build suffix array
sa = build_suffix_array(text)

print("Text:", text)
print("\nSuffix Array:")
print("Index | SA[i] | Suffix")
print("-" * 40)
for i, pos in enumerate(sa):
    print(f"{i:5d} | {pos:5d} | {text[pos:]}")

In [ ]:
# Build BWT
bwt = build_bwt(text, sa)

print("Original text:", text)
print("BWT:          ", bwt)
print("\nBWT is a permutation of the original text:")
print("Sorted original:", ''.join(sorted(text)))
print("Sorted BWT:     ", ''.join(sorted(bwt)))

## 3. Pattern Matching Examples

Now let's demonstrate pattern matching using both suffix array and FM-index.

In [ ]:
# Search for patterns using suffix array
patterns = ["ana", "na", "ban", "xyz"]

print("Suffix Array Search Results:")
print("-" * 40)
for pattern in patterns:
    positions = sa_find_all(text, sa, pattern)
    if positions:
        print(f"Pattern '{pattern}' found at positions: {positions}")
    else:
        print(f"Pattern '{pattern}' not found")

In [ ]:
# Build FM-index structures
c_table = build_c_table(bwt)
occ = build_occ_table(bwt)

print("FM-Index Search Results:")
print("-" * 40)
for pattern in patterns:
    positions = fm_find_all(pattern, text, sa, bwt, c_table, occ)
    if positions:
        print(f"Pattern '{pattern}' found at positions: {positions}")
    else:
        print(f"Pattern '{pattern}' not found")

print("\nBoth methods produce identical results!")

## 4. Runtime Experiments

Now we'll conduct performance experiments comparing SA search and FM search with:
- Text size: **1,000,000 characters**
- Pattern length: **100 characters**
- Multiple random patterns

**Note:** We exclude the build time and only measure search time.

In [ ]:
# Generate a large random text
def generate_random_text(length, alphabet="abcdefghijklmnopqrstuvwxyz"):
    """Generate a random text of given length."""
    return ''.join(random.choice(alphabet) for _ in range(length)) + '$'

# Generate random patterns from text
def generate_patterns_from_text(text, num_patterns, pattern_length):
    """Generate random patterns that exist in the text."""
    patterns = []
    max_start = len(text) - pattern_length - 1  # -1 for sentinel
    for _ in range(num_patterns):
        start = random.randint(0, max_start)
        patterns.append(text[start:start + pattern_length])
    return patterns

print("Generating test data...")
random.seed(42)
text_size = 1_000_000
pattern_length = 100
num_patterns = 20

# Generate text
large_text = generate_random_text(text_size, alphabet="abcdefgh")
print(f"Generated text of length {len(large_text)}")

# Generate patterns
patterns = generate_patterns_from_text(large_text, num_patterns, pattern_length)
print(f"Generated {num_patterns} patterns of length {pattern_length}")

In [ ]:
# Build data structures (excluded from timing)
print("Building suffix array...")
start = time.time()
large_sa = build_suffix_array(large_text)
sa_build_time = time.time() - start
print(f"SA build time: {sa_build_time:.2f} seconds")

print("\nBuilding BWT and FM-index structures...")
start = time.time()
large_bwt = build_bwt(large_text, large_sa)
large_c_table = build_c_table(large_bwt)
large_occ = build_occ_table(large_bwt)
fm_build_time = time.time() - start
print(f"BWT/FM-index build time: {fm_build_time:.2f} seconds")
print(f"Total build time: {sa_build_time + fm_build_time:.2f} seconds")

In [ ]:
# Measure search times (excluding build time)
print("\nMeasuring search performance...")
print("=" * 60)

sa_search_times = []
fm_search_times = []
match_counts = []

for i, pattern in enumerate(patterns):
    # SA search
    start = time.time()
    sa_results = sa_find_all(large_text, large_sa, pattern)
    sa_time = time.time() - start
    sa_search_times.append(sa_time)
    
    # FM search
    start = time.time()
    fm_results = fm_find_all(pattern, large_text, large_sa, large_bwt, large_c_table, large_occ)
    fm_time = time.time() - start
    fm_search_times.append(fm_time)
    
    # Verify results match
    assert sa_results == fm_results, f"Results mismatch for pattern {i}"
    match_counts.append(len(sa_results))
    
    if i < 5:  # Print first 5 results
        print(f"Pattern {i+1}: {len(sa_results)} matches | SA: {sa_time*1000:.3f}ms | FM: {fm_time*1000:.3f}ms")

print("\nSummary Statistics (in milliseconds):")
print("=" * 60)
print(f"Suffix Array Search:")
print(f"  Mean: {np.mean(sa_search_times)*1000:.3f}ms")
print(f"  Median: {np.median(sa_search_times)*1000:.3f}ms")
print(f"  Min: {np.min(sa_search_times)*1000:.3f}ms")
print(f"  Max: {np.max(sa_search_times)*1000:.3f}ms")

print(f"\nFM-Index Search:")
print(f"  Mean: {np.mean(fm_search_times)*1000:.3f}ms")
print(f"  Median: {np.median(fm_search_times)*1000:.3f}ms")
print(f"  Min: {np.min(fm_search_times)*1000:.3f}ms")
print(f"  Max: {np.max(fm_search_times)*1000:.3f}ms")

speedup = np.mean(sa_search_times) / np.mean(fm_search_times)
print(f"\nAverage speedup (SA/FM): {speedup:.2f}x")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Search times comparison
x = range(1, len(patterns) + 1)
axes[0].plot(x, [t*1000 for t in sa_search_times], 'o-', label='SA Search', linewidth=2, markersize=6)
axes[0].plot(x, [t*1000 for t in fm_search_times], 's-', label='FM Search', linewidth=2, markersize=6)
axes[0].set_xlabel('Pattern Number', fontsize=12)
axes[0].set_ylabel('Search Time (ms)', fontsize=12)
axes[0].set_title('Search Time Comparison (100-char patterns on 1M text)', fontsize=13)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot 2: Box plot comparison
axes[1].boxplot([np.array(sa_search_times)*1000, np.array(fm_search_times)*1000],
                labels=['SA Search', 'FM Search'])
axes[1].set_ylabel('Search Time (ms)', fontsize=12)
axes[1].set_title('Search Time Distribution', fontsize=13)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('runtime_experiments.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved as 'runtime_experiments.png'")

## 5. Analysis

### Key Observations:

1. **FM-index is typically faster for search operations**: FM-index has O(m + k) time complexity for pattern matching, which is faster than the O(m log n + k) complexity of binary search on suffix arrays.

2. **Trade-offs**:
   - **SA Search**: Simpler to implement, lower memory overhead
   - **FM Search**: Faster queries, but requires additional space for C-table and Occ table (O(σn))

3. **When to use each**:
   - Use **SA search** when memory is limited or for small alphabets
   - Use **FM-index** when query speed is critical and memory is available

4. **Build time is excluded** from the experiments as specified. Both structures are pre-built and only search performance is measured.

## 6. Conclusion

This notebook demonstrated:

1. **Suffix Array Construction** using prefix-doubling algorithm (O(n log² n))
2. **Binary Search** on suffix arrays for pattern matching
3. **BWT and FM-index** construction and usage
4. **Performance comparison** showing FM-index advantages for search operations
5. **Correctness verification** through equivalence testing

Both approaches are powerful tools for exact pattern matching in large texts, with different trade-offs between simplicity, memory usage, and query speed.